In [1]:
import os
from numpy import geomspace
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Basics.DataStructures import Vector

In [4]:
viscosity_vals = geomspace(2.54e-6, 0.0254, 80)  # Varying viscosity values
reynolds_no = 1. * 0.0254 / viscosity_vals 

### Create Case Directories

In [5]:
# Genererate cases with different viscosities
for i, Re in enumerate(reynolds_no):
    # === Laminar case ===
    if Re < 2000:
        templateDir = SolutionDirectory("Templates/backwardStepLaminarTemplate", archive=None, paraviewLink=False)
    # === Turbulent case ===
    else:
        templateDir = SolutionDirectory("Templates/backwardStepTurbulentTemplate", archive=None, paraviewLink=False)
    # Create a new case
    case = templateDir.cloneCase(f"backwardStepCases/backwardStepFlow-Re{Re:.2f}", paraviewLink=False)
    # Get the directory with viscosity parameter
    transportProps = ParsedParameterFile(os.path.join(case.name, "constant", "transportProperties"))
    # Set appropriate value of viscosity
    transportProps["nu"] = viscosity_vals[i]
    # Save the transport properties dictionary
    transportProps.writeFile()

### Export graphs for ML

In [2]:
import pyvista as pv

In [5]:
# Generate pressure and velocity contours and pathlines for each case in poiseuilleFlow dir
case_dir_list = os.listdir("backwardStepCases/")
for case in case_dir_list:
    # Define a path to a folder
    case_path = os.path.join("backwardStepCases/", case)
    # Get the path to VTK folder
    vtk_path = os.path.join(case_path, "VTK")
    # Get the list of appropriate files
    vtk_files = os.listdir(vtk_path)
    # Get only the files with .vtm extension
    vtk_files = [a_file for a_file in vtk_files if ".vtm" in a_file and ".series" not in a_file]
    # Get the last iteration
    vtk_file = max(vtk_files, key=lambda f: int(f.split("_")[-1].split(".")[0]))
    # Create path to the VTK file
    vtk_file_path = os.path.join(vtk_path, vtk_file)
    # Get the Reynolds No.
    Re_num = vtk_file.split("-")[-1].split("_")[0]
    # Define appropriate camera position
    camera_position = [(0.1, 0, 0.5), (0.1, 0, 0), (0, 0, 0)]
    # Read mesh
    mesh = pv.read(vtk_file_path)
    vol = mesh[0]
    # Streamline plot
    combined_mesh = mesh.combine()
    combined_mesh.set_active_vectors("U")
    one_streamline = combined_mesh.streamlines(
        pointa=(0.1, -0.0254, 0),
        pointb=(0.1, 0.0254, 0),
        n_points=15,
        max_length=1,
        compute_vorticity=False,  # vorticity already exists in dataset
    )
    # Create a plotter
    plotter = pv.Plotter(off_screen=True)
    windows_size = (500, 300)
    # Pressure field plot
    plotter.add_mesh(vol, scalars="p", cmap="linear_grey_0_100_c0", show_scalar_bar=False)
    plotter.camera_position = camera_position
    plotter.window_size = windows_size
    plotter.screenshot(filename=f"mlDataset/backwardStep/images/p_field{Re_num}.png")
    plotter.clear()
    # Velocity field plot
    plotter.add_mesh(vol, scalars="U", cmap="linear_grey_0_100_c0", show_scalar_bar=False)
    plotter.screenshot(filename=f"mlDataset/backwardStep/images/U_field{Re_num}.png")
    plotter.clear()
    # Streamline plot
    plotter.add_mesh(one_streamline, color="black")
    plotter.screenshot(filename=f"mlDataset/backwardStep/images/streamlines{Re_num}.png")
    plotter.close()

### Create a mask

In [ ]:
plotter = pv.Plotter()
plotter.add_mesh(mesh[0], color="white", show_edges=False)
plotter.camera_position = camera_position
plotter.background_color = 'black'
plotter.window_size = (500, 300)
plotter.screenshot(filename=f"mlDataset/backwardStep/mask/backwardStepFlowMask.png")

pyvista_ndarray([[[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 ...,

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0]],

                 [[0, 0, 0],
                  [0, 0, 0],
                  [0, 0, 0],
                  ...,
                  [0, 0, 0],
                  [0, 0, 0],
             